Data

Training data: https://github.com/ga-students/DS_HK_1/blob/gh-pages/data/class/lemonsClassificationData/lemon_training.csv

Data dictionary: https://github.com/ga-students/DS_HK_1/blob/gh-pages/data/class/lemonsClassificationData/description.txt

Task

Create a Gradient Boosted Classifier

Predict whether a automobile will be a lemon based on certain characteristics.

Use cross validation

What's the highest accuracy you can get on the test dataset?

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [54]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

import us

In [72]:
df_lemons = pd.read_csv('lemon_training.csv', na_filter=False)
df_lemons.drop(list(df_lemons.filter(regex = 'MMR')), axis = 1, inplace = True)
# change states into numbers
df_lemons['VNST'] = df_lemons['VNST'].map({
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'}).astype('int64')
df_lemons['Make'] = df_lemons['Make'].map({'MAZDA' : 0, 'DODGE' : 1, 'FORD' : 2, 'MITSUBISHI' : 3, 'KIA' : 4, 'CHEVROLET' : 5,
       'SATURN' : 6, 'GMC' : 7, 'MERCURY' : 8, 'HYUNDAI' : 9, 'CHRYSLER' : 10, 'NISSAN' : 11,
       'PONTIAC' : 12, 'JEEP' : 13, 'SUZUKI' : 14, 'HONDA' : 15, 'OLDSMOBILE' : 16, 'TOYOTA' : 17,
       'BUICK' : 18, 'SCION' : 19, 'VOLKSWAGEN' : 20, 'ISUZU' : 21, 'LINCOLN' : 22, 'MINI' : 23,
       'SUBARU' : 24, 'CADILLAC' : 25, 'VOLVO' : 26, 'INFINITI' : 27, 'LEXUS' : 28, 'ACURA' : 29,
       'TOYOTA SCION' : 30, 'PLYMOUTH' : 31}).astype('int64')
df_lemons.head()

C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,Size,TopThreeAmericanName,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,0,MAZDA3,i,4D SEDAN I,...,MEDIUM,OTHER,,,21973,33619,12,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,1,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,LARGE TRUCK,CHRYSLER,,,19638,33619,12,7600.0,0,1053
2,5,0,12/7/2009,ADESA,2005,4,2,FOCUS,ZX3,2D COUPE ZX3,...,COMPACT,FORD,,,19638,33619,12,4000.0,0,1020
3,6,0,12/7/2009,ADESA,2004,5,3,GALANT 4C,ES,4D SEDAN ES,...,MEDIUM,OTHER,,,19638,33619,12,5600.0,0,594
4,7,0,12/7/2009,ADESA,2004,5,4,SPECTRA,EX,4D SEDAN EX,...,MEDIUM,OTHER,,,19638,33619,12,4200.0,0,533


In [73]:
df_lemons.Model.unique()

array(['MAZDA3', '1500 RAM PICKUP 2WD', 'FOCUS', 'GALANT 4C', 'SPECTRA',
       'CARAVAN GRAND FWD V6', 'CAVALIER 4C', 'TRAILBLAZER 2WD 6C',
       'VUE 2WD 4C', 'ENVOY XL 2WD 6C', 'MONTE CARLO', 'HHR', 'SABLE',
       'ENVOY XL 4WD 6C', 'SUBURBAN 1500 2WD', 'MUSTANG V6',
       'AVALANCHE 1500 2WD V', 'WINDSTAR FWD V6', 'SONATA V6', '300',
       'SENTRA', 'PACIFICA FWD', 'IMPALA V6', 'G6 V6', 'ION', 'NEON',
       'DURANGO 2WD V8', 'FIVE HUNDRED', 'GRAND PRIX', 'SEBRING V6',
       'LIBERTY 2WD V6', 'FORENZA', 'TAURUS', 'DAKOTA PICKUP 2WD V6',
       'TAHOE 2WD', 'EXPEDITION 2WD V8', 'ELEMENT 2WD', 'SUNFIRE',
       'FREESTYLE 2WD V6', 'F150 PICKUP 2WD V8', 'ALTIMA',
       'TRAILBLAZER EXT 2WD', 'PT CRUISER', 'DURANGO 2WD V6', 'IMPALA',
       'ALERO 4C', 'CAMRY V6', 'EQUINOX FWD V6', 'MAZDA6', 'RENO',
       'SILHOUETTE', 'SEBRING 4C', 'RANGER PICKUP 2WD V6', 'G6 4C',
       'ESCAPE 4WD V6', 'ENVOY 2WD 6C', 'L SERIES', 'UPLANDER FWD V6',
       'ELANTRA', 'COBALT', 'EXPLORER 2WD V6

In [74]:
df_lemons.dtypes

RefId                     int64
IsBadBuy                  int64
PurchDate                object
Auction                  object
VehYear                   int64
VehicleAge                int64
Make                      int64
Model                    object
Trim                     object
SubModel                 object
Color                    object
Transmission             object
WheelTypeID              object
WheelType                object
VehOdo                    int64
Nationality              object
Size                     object
TopThreeAmericanName     object
PRIMEUNIT                object
AUCGUART                 object
BYRNO                     int64
VNZIP1                    int64
VNST                      int64
VehBCost                float64
IsOnlineSale              int64
WarrantyCost              int64
dtype: object

In [75]:
nominal_cols = ['Auction', 'Trim', 'TopThreeAmericanName', 'Model', 'SubModel', 'Color', 'Transmission', 'WheelType', 'WheelTypeID',
                'PRIMEUNIT', 'AUCGUART', 'Nationality', 'Size']

num_cols = ['VehicleAge', 'VehOdo', 'BYRNO', 'VNZIP1', 'IsOnlineSale', 'WarrantyCost', 'VNST', 'Make']
                        

def fill_missing_values(df):
    '''
    This function fills in the missing values
    Currently it's a simple solution
    - Mode for nominal columns
    - Median for numerical columns
    '''

    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())
    
    for col in nominal_cols:
        mode = df[col].mode()[0]
        df[col] = df[col].fillna(mode)

    return df

In [76]:
df_lemons.columns

Index(['RefId', 'IsBadBuy', 'PurchDate', 'Auction', 'VehYear', 'VehicleAge',
       'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission',
       'WheelTypeID', 'WheelType', 'VehOdo', 'Nationality', 'Size',
       'TopThreeAmericanName', 'PRIMEUNIT', 'AUCGUART', 'BYRNO', 'VNZIP1',
       'VNST', 'VehBCost', 'IsOnlineSale', 'WarrantyCost'],
      dtype='object')

In [77]:
df_filled = fill_missing_values(df_lemons)
df_filled.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,Size,TopThreeAmericanName,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,0,MAZDA3,i,4D SEDAN I,...,MEDIUM,OTHER,,,21973,33619,12,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,1,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,LARGE TRUCK,CHRYSLER,,,19638,33619,12,7600.0,0,1053
2,5,0,12/7/2009,ADESA,2005,4,2,FOCUS,ZX3,2D COUPE ZX3,...,COMPACT,FORD,,,19638,33619,12,4000.0,0,1020
3,6,0,12/7/2009,ADESA,2004,5,3,GALANT 4C,ES,4D SEDAN ES,...,MEDIUM,OTHER,,,19638,33619,12,5600.0,0,594
4,7,0,12/7/2009,ADESA,2004,5,4,SPECTRA,EX,4D SEDAN EX,...,MEDIUM,OTHER,,,19638,33619,12,4200.0,0,533


In [68]:
y=df_filled["IsBadBuy"]
X=df_filled.drop(['IsBadBuy', 'Auction', 'Trim', 'agrossincome', 'native-country', 'sex', 'marital-status', 'over50k'], axis=1)
X.shape


(51088, 25)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [70]:
model = GradientBoostingRegressor().fit(X_train, y_train)

ValueError: could not convert string to float: 